# HDS5210-2022 Fall Midterm

In the midterm exercise, you're going to use all the programming and data management skills you've developed so far to build a pricing calculator that will calculate how much money should be reimbursed for the visits in a CSV file. To do this, you will need to get allowed amounts (aka rates) from a JSON file, apply some special rules, and then calculate various totals by hospital or by month.

Each step of the midterm will build up to form your final solution. Along the way, I've provided plenty of test cases to make sure you're getting each step correct.

All functions require docstrings with a description and at least one test case.

The midterm is due Monday, October 24th at 11:59 PM CST.

---

## Step 1: Average Rate

With the `/data/negotiated_rates.json` file as input for your first function, read all the `allowed_amount` attributes and calculate an average allowed amount over all rates in the file.

Your function should be named **average_rate()**, take the file's name as it's input parameter, and return a float as the result.

**ROUND YOUR ANSWER TO 2 DECIMAL PLACES**


In [1]:
import json
def average_rate(file):
    """ finding Average of allowed amount over all rates in the negotiated_rates file
    
    >>> assert(average_rate('/data/negotiated_rates.json')
    38.67
    """
    total = 0
    count = 0
    with open('/data/negotiated_rates.json', 'r') as F:
        n_rate = json.load(F)
        for amount in n_rate['out_of_network']:
            for amt in amount['allowed_amounts']:
                total +=amt['payments']['allowed_amount']
                count += 1
        average = round(total/count, 2)
        return average

In [2]:
assert(average_rate('/data/negotiated_rates.json') == 38.67)

---

## Step 2: Rate for a Billing Code / Service Code Combination

For the next step, we need to be able to look up the allowed amount for any given billing code / service code combination.

In this data, the billing code represents the service or procedure that was provided. The service code represents the type of site where the service was provided.

Your function should be named **get_rate()** and take three parameters: the JSON file name, the billing code, and the service code. If your code can't find that combination in the file, it should return None.

In [3]:
import json

def get_rate(file, billing_code, service_code):
    """finding rate for a billing code and service code combination using three parameters
       JSON file name, the billing code, and the service code.
    
    
    
    >>> assert (get_rate('/data/negotiated_rates.json','G0283','11')
    8.78
    """
    with open('/data/negotiated_rates.json', 'r') as F:
        nr = json.load(F)

        for i in nr['out_of_network']:
            bi_c = i['billing_code']

            try:
                if bi_c == billing_code:            
                    for j in i['allowed_amounts']:
                        si_c = j['service_code']

                        if service_code == si_c:
                            return j['payments']['allowed_amount']
            except:
                return None


In [4]:
assert (get_rate('/data/negotiated_rates.json','G0283','11') == 8.78)
assert (get_rate('/data/negotiated_rates.json','97140','11') == 20.03)
assert (get_rate('/data/negotiated_rates.json','97110','12') == 26.62)

---

## Part 3: Special Rules for Rates

There are some special rules for adjusting rates depending on a patient's age and the day of week.  (These are made up. Not from the real world.) In the next part of the midterm, you'll need to create a function that applies these extra rules to a rate and return the adjusted value.

1. If the day of week is Monday, charge only 75% of the allowed amount.
2. If the patient's age is 65 or higher, charge only 50% of the allowed amount.
3. If's both Monday and the patient's age is 65 or higher, charge only 50% of the allowed amount.
4. If neither of these conditions are true, charge the whole amount.

Your function should be named **get_adjusted_rate()** and take five parameters: file name, billing code, service code, patient age, and visit date.  Your function should return the adjusted rate (based on the rules above) or None if the rate couldn't be found in the file.

Note that your function will take a date in the form `%Y-%m-%d` ([see datetime.strptime()](https://docs.python.org/3/library/datetime.html#datetime.datetime.strptime)) and will need to calculate the [day of week](https://docs.python.org/3/library/datetime.html#datetime.date.weekday).

**ROUND YOUR ANSWER TO 2 DECIMAL PLACES**

In [5]:
from datetime import datetime

def get_adjusted_rate(file, billing_code, service_code, patient_age, visit_date):
    """
    this program gets adjusted rate using five parameters: file name, billing code, service code, patient age, and visit date
    
    
    >>> assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-03')
    4.39
    """
    adjusted_rate = get_rate(file, billing_code, service_code)
    
    dt = datetime.strptime(visit_date, "%Y-%m-%d")
    week_d = dt.strftime('%A')

    try:
        if week_d == 'Monday' and patient_age >= 65:
             adjusted_rate = 0.5 *  adjusted_rate

        elif week_d == 'Monday':                
             adjusted_rate = 0.75 *  adjusted_rate

        elif patient_age >= 65:
             adjusted_rate = 0.5 *  adjusted_rate
                
        else:
            return  adjusted_rate
        return round( adjusted_rate, 2)
    except:
        return None


In [6]:

assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-07') == 8.78)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-03') == 6.58)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-03') == 4.39)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-08') == 4.39)

---

## Step 4: Calculate total payments for a list of visits

Last step, we're going to put your get_adjusted_rate() function to work on a list of visits from a few hospitals in the St. Louis region.

Your last function, **summarize_reimbursement()** needs to read an encounter file and the negotiated rates file, compute an adjusted rate for each encounter (row) in the input file, and return two dictionaries of information:
1. Total expected reimbursement by month
2. Total expected reimbursement by hospital

Your should be able to run your function as `by_month, by_hospital = summarize_reimbursement(visits, negotiated_rates)` and have the two answers below:

**by_month**
```json
{
    '2021-05': 192.38,
    '2021-03': 378.72,
    '2021-07': 277.67,
    '2021-06': 236.53,
    '2021-11': 229.7,
    '2021-10': 234.52,
    '2021-12': 297.87,
    '2021-04': 337.7,
    '2021-09': 160.4,
    '2021-01': 111.91,
    '2021-02': 158.55,
    '2021-08': 152.28
}
```

**by_hospital**
```json
{
    'Missouri Baptist': 514.18,
    'SSM DePaul': 460.02,
    'SLU Hospital': 409.67,
    'Barnes Jewish': 485.7,
    'Mercy Springfield': 518.59,
    'Mercy St. Louis': 380.07
}
```

**Round your totals to 2 decimal places**

**If rates are not found, just ignore them**

In [7]:
import json
import csv
from datetime import datetime

def summarize_reimbursement(visits, negotiated_rates):
    """ this program Total adjusted rate by month or hospital name
    
    
    >>> assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy Springfield']
    518.59
    """
    by_month = {}
    by_hospital = {}
    with open(visits, 'r') as cf:
        _csv = csv.DictReader(cf)
        for row in _csv:
            bi_c = row['BILLING_CODE']
            si_c = row['SERVICE_CODE']
            paitient_age = row['PATIENT_AGE']
            v_d = row['VISIT_DATE']
            hp = row['HOSPITAL']
            date = datetime.strptime(v_d, "%Y-%m-%d")
            m = date.strftime('%Y-%m') #month

            rt = get_adjusted_rate(
                negotiated_rates, bi_c, si_c, int(paitient_age), v_d
            )

            #create by_month dictionary
            if m in by_month:
                adj_rate = by_month[m]
                if rt is not None:                  
                    by_month[m] = round(adj_rate + rt, 2)
                else:
                    rt = 0
                    by_month[m] = round(adj_rate + rt, 2)
            else:
                if rt is not None:
                    by_month[m] = round(rt, 2)
                else:
                    by_month[m] = 0
                    
            
            if hp in by_hospital:
                adj_rate = by_hospital[hp]
                if rt is not None:                  
                    by_hospital[hp] = round(adj_rate + rt, 2)
                else:
                    rt = 0
                    by_hospital[hp] = round(adj_rate + rt, 2)
            else:
                if rt is not None:
                    by_hospital[row['HOSPITAL']] = round(rt, 2)
                else:
                    by_hospital[row['HOSPITAL']] = 0
        
        return by_month, by_hospital


In [8]:
by_month, by_hospital = summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')

In [9]:
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-07'] == 277.67)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-03'] == 378.72)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy St. Louis'] == 380.07)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy Springfield'] == 518.59)